In [ ]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.get_device_name(0))


# 查看预测目标内部关系

In [ ]:
# 📦
import os
import gc
import cv2
import h5py
import json
import time
import shutil
import socket
import psutil
import subprocess
from pathlib import Path
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

from tqdm import tqdm
from tabulate import tabulate

from sklearn.model_selection import KFold, GroupKFold
from sklearn.metrics import r2_score

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.amp import autocast, GradScaler       

import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image

import timm
from torchvision import models, transforms
from torchvision.models import get_model_weights


d:\Software\conda\envs\kaggle2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 🌱 Path Initialization
if socket.gethostname() == 'hao-2':
    dir = Path('D:/DATA_hao/Kaggle_/csiro-biomass/')
    DIRS = {
        "dir"  : dir,
        "train": Path(dir, "train"),
        "test" : Path(dir, "test"),
        "model": Path(dir, "DualStream_multihead"),
        "data" : Path(dir),
    }


    print("✅ file path ：")
    for key, path in DIRS.items():
        print(f"{key:<12} : {path}")

elif socket.gethostname() == 'user-PowerEdge-XE9680':
    dir = Path('/data4/huangweigang/gh/csiro-biomass')
    DIRS = {
        "dir"  : dir,
        "train": Path(dir, "train"),
        "test" : Path(dir, "test"),
        "model": Path(dir, "DualStream_multihead"),
        "data" : Path(dir),
    }


    print("✅ file path ：")
    for key, path in DIRS.items():
        print(f"{key:<12} : {path}")

else:
    dir = Path('/kaggle/input/csiro-biomass')
    DIRS = {
        "dir"  : dir,
        "train": Path(dir, "train"),
        "test" : Path(dir, "test"),
        "model": Path('/kaggle/input', "dualstream-multihead2025-11-04-02-07-11"),
        "data" : Path("/kaggle/working/"),
    }


    print("✅ file path ：")
    for key, path in DIRS.items():
        print(f"{key:<12} : {path}")


✅ file path ：
dir          : D:\DATA_hao\Kaggle_\csiro-biomass
train        : D:\DATA_hao\Kaggle_\csiro-biomass\train
test         : D:\DATA_hao\Kaggle_\csiro-biomass\test
model        : D:\DATA_hao\Kaggle_\csiro-biomass\DualStream_multihead
data         : D:\DATA_hao\Kaggle_\csiro-biomass


In [3]:
# Helper functions
def show_df_info(df, name: str):
    """
    Print the shape and column names of a single DataFrame.

    Args:
        df   : pandas.DataFrame
        name : Display name (string)
    """
    print(f"📊 {name:<16} shape: {str(df.shape):<16}  columns: {df.columns.tolist()}")

def select_free_gpu(threshold_mem_MB = 500, threshold_util = 20):
    """
    Automatically select an available GPU (works in both .py and Jupyter environments).
    """

    # === Internal function: query GPU info from nvidia-smi ===
    def get_gpu_info():
        """Retrieve GPU information using nvidia-smi."""
        query = (
            "index,name,memory.used,memory.total,utilization.gpu,temperature.gpu,power.draw"
        )
        result = subprocess.run(
            ["nvidia-smi", f"--query-gpu={query}", "--format=csv,noheader,nounits"],
            capture_output=True, text=True
        )

        gpus = []
        for line in result.stdout.strip().split("\n"):
            idx, name, mem_used, mem_total, util, temp, power = [x.strip() for x in line.split(",")]
            gpus.append({
                "index": int(idx),
                "name": name,
                "mem_used_MB": int(mem_used),
                "mem_total_MB": int(mem_total),
                "util_%": int(util),
                "temp_C": int(temp),
                "power_W": float(power),
            })
        return gpus

    # === Main logic ===
    gpus = get_gpu_info()

    # Select GPUs with low memory and utilization
    free_gpus = [
        g for g in gpus
        if g["mem_used_MB"] < threshold_mem_MB and g["util_%"] <= threshold_util
    ]

    if not free_gpus:
        gpus.sort(key=lambda x: x["mem_used_MB"])
        selected = gpus[0]
        reason = "(No fully idle GPU found — selected the one with lowest memory usage)"
    else:
        selected = free_gpus[0]
        reason = "(Idle GPU detected)"

    # Print GPU information table
    print(tabulate(
        [[g["index"], g["name"], f"{g['mem_used_MB']}/{g['mem_total_MB']} MB",
          f"{g['util_%']}%", f"{g['temp_C']}°C", f"{g['power_W']}W"]
         for g in gpus],
        headers=["GPU", "Name", "Memory", "Util", "Temp", "Power"],
        tablefmt="grid"
    ))

    idx = selected["index"]
    device_name = f"cuda:{idx}"

    # Detect if running inside a Jupyter Notebook
    try:
        from IPython import get_ipython
        in_notebook = get_ipython() is not None
    except Exception:
        in_notebook = False

    if not in_notebook:
        # ✅ Safe to set environment variable in normal Python script
        os.environ["CUDA_VISIBLE_DEVICES"] = str(idx)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"\n✅ Selected GPU {idx} {reason}")
        print(f"Current device: {device} (logical GPU {idx})\n")
    else:
        # ⚠️ In notebook environments, do not modify environment variables
        device = torch.device(device_name if torch.cuda.is_available() else "cpu")
        print(f"\n⚠️ Detected Jupyter environment — not modifying CUDA_VISIBLE_DEVICES.")
        print(f"✅ Using device: {device_name} {reason}\n")

    return idx, device


In [4]:
# Load and preprocess training data
def load_and_prepare_train_df():
    # 1️⃣ Read raw CSV file
    df_file_path = Path(DIRS["dir"]) / "train.csv"
    df = pd.read_csv(df_file_path)

    # 2️⃣ Extract unique ID (e.g., "ID1011485656__Dry_Green_g" → "ID1011485656")
    df["ID"] = df["sample_id"].str.split("__").str[0]

    # 3️⃣ Move ID column to the front
    df = df[["ID"] + [c for c in df if c != "ID"]]

    # 4️⃣ Pivot target values (long → wide format)
    df_targets = (
        df.pivot_table(
            index="ID",
            columns="target_name",
            values="target",
            aggfunc="first"
        )
        .reset_index()
    )
    df_targets.columns.name = None  # remove multi-index column names

    # 5️⃣ Extract metadata (one row per ID)
    meta_cols = [
        "ID", "image_path", "Sampling_Date", "State",
        "Species", "Pre_GSHH_NDVI", "Height_Ave_cm"
    ]
    df_meta = df[meta_cols].drop_duplicates(subset="ID")

    # 6️⃣ Merge metadata with target values
    df_train = pd.merge(df_meta, df_targets, on="ID", how="left")
    show_df_info(df_train, "df_train")

    return df_train


In [ ]:
# 读取训练数据
df = load_and_prepare_train_df()

# 最终报告创建新特征


In [16]:
df_train = load_and_prepare_train_df().copy()

print("🌿 Cleaning species names ...")
df_train["Species"] = df_train["Species"].astype(str).str.lower().str.strip()

# --- 按下划线分割，并展开 ---
all_species = (
    df_train["Species"]
    .str.split("_")         # 分割
    .explode()              # 展开成新行
    .str.strip()            # 去空格
)

# --- 统计出现次数 ---
species_counts = (
    all_species.value_counts()
    .reset_index()
    .rename(columns={"index": "Species", "Species": "Count"})
)

print("\n📊 Species frequency:")
print(species_counts)


📊 df_train         shape: (357, 12)         columns: ['ID', 'image_path', 'Sampling_Date', 'State', 'Species', 'Pre_GSHH_NDVI', 'Height_Ave_cm', 'Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']
🌿 Cleaning species names ...

📊 Species frequency:
                Count  count
0              clover    207
1            ryegrass    175
2            phalaris     76
3              fescue     38
4             lucerne     22
5         barleygrass     18
6         silvergrass     11
7            capeweed     11
8          speargrass     11
9           crumbweed     10
10        whiteclover     10
11         bromegrass      7
12      subcloverlosa      5
13  subcloverdalkeith      3
14              mixed      2
